### Data Processing

This file contains scripts relevant for cleaning and processing datasets used for the Hex2Vec classifier

In [5]:
output_directory = "/home/janelle9/land_use_framework/data/" #where we save our processed shp "tags"
input_directory = "/home/janelle9/Japan-GIS-Data/"

# JAXA Land Cover

For processing JAXA raster data, data is available for download via https://www.eorc.jaxa.jp/ALOS/en/dataset/lulc_e.htm#download

# Converts Raster files to Shapefiles

This file contains code snippets for processing shapefile data for the Hex2Vec pipeline. This process can be applied to raster data. Raster data can be converted to vector data using the process below: 

### Prequisites

#### Gdal Installation
pip install GDAL (if this doesn't work try the anaconda environment below)

#### Anaconda Installation

- curl -O https://repo.anaconda.com/archive/Anaconda3-<INSTALLER_VERSION>-Linux-x86_64.sh <br>
    - (List of Anaconda versions: https://repo.anaconda.com/archive/, I used 2024.02-1) <br>
- bash ~/Downloads/Anaconda3-<INSTALLER_VERSION>-Linux-x86_64.sh <br>
- Press and hold Enter to review the license agreement
- Enter yes
- Press enter to accept default install location or speciyf an alternate directory

- eval "$(/<ANACONDA_INSTALLATION_FOLDER>/bin/conda activate hook)"
- export PATH="<ANACONDA_INSTALLATION_FOLDER>/anaconda3/bin:$PATH
    *  Optionally, you can add this command to .bashrc 

##### Creating the Environment
- conda create --name condaEnv python=3.9.6
- conda install -c conda-forge gdal
- gdalinfo --version (used to verify the installation)

Install ipykernel and requirements.txt as needed

Step 1) Convert Raster files to Polygon file using gdal_polygonize command. A bash script is available in ~/Japan-GIS-Data/landcover_data/convert_tif.sh

Step 2) Read in raw shapefiles and perform mappings to resemble OSM tags

Step 3) Add unique identifier

Step 4) Save cleaned data

In [ ]:
#REQUIRES GDAL COMPATIBLE ENVIRONMENT
from osgeo import gdal, ogr
import geopandas as gpd
import os

#(1) reading in raster data
shp = gpd.read_file("/home/janelle9/Japan-GIS-Data/landcover_data/shapefile/JAXA_HRLULC_Japan_v23.12_250m.shp") #Reading in xxx m Landcover data

#(2) Land Cover Mapping Function
land_cover_labels = ["NULL", "Water", "Built-up", "Paddy Field", "Cropland", "Grassland", "DBF", "DNF", "EBF", "ENF", "Bare", "Bamboo Forest", "Solar Panel", "Wetland", "Greenhouse"]
def land_cover_mapping(value):
    return land_cover_labels[value]

#Converting to OSM tag
#(3) add index
shp['feature_id'] = shp.index

#Convert mappings
shp["land_cover"] = shp['DN'].apply(land_cover_mapping)
shp = shp[['feature_id', 'land_cover', 'geometry']]

shp.set_index('feature_id', inplace=True)

#(4) Save new shapefile
shp.to_file(output_directory + "lc_250_tags")

## Slope and Aspect

Cleans data from Elevation and Slope 5th Order Mesh Dataset, data can be wondloaded via ~/Japan-GIS-Data/slope/download.sh and extracted via ~/Japan-GIS-Data/slope/unzip.sh.

These scripts can be reworked to download any of the data from the National Land Numerical Information Download Site

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

#read in all files from dataset

path = "/home/janelle9/Japan-GIS-Data/slope/"

#concatenates all shapefiles in specified directory
file = os.listdir(path)
path = [os.path.join(path, i) for i in file if ".shp" in i]
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)
gdf.crs = 'EPSG:4326'


#create new dataframe, converts values to numeric and discretize by degrees of 5
slope = gpd.GeoDataFrame(columns=['avg_slope', 'max_slope', 'min_slope', 'max_aspect', 'min_aspect', 'geometry'], crs='EPSG:4326')
slope['avg_slope'] = np.floor(pd.to_numeric(gdf['G04d_010'], errors='coerce') / 5)
slope['max_slope'] = np.floor( pd.to_numeric(gdf['G04d_006'], errors='coerce') / 5)
slope['min_slope'] = np.floor (pd.to_numeric(gdf['G04d_009'], errors='coerce') / 5) 
slope['max_aspect'] = pd.to_numeric(gdf['G04d_007'], errors='coerce')  # Codelist type (0 = no direction, 1 = north, 2 = northeast, 3 = east, 4 = southeast, 5 = south, 6 = southwest, 7 = west, 8 = northwest)
slope['min_aspect'] = pd.to_numeric(gdf['G04d_009'], errors='coerce')
slope['geometry'] = gdf['geometry']
slope = slope.set_geometry('geometry')
slope.crs = 'EPSG:4326'
slope = slope.dropna() #clean out null values

slope['feature_id'] = slope.index
slope.set_index('feature_id', inplace=True)

slope.to_file(output_directory + "slope_aspect_tags")


In [ ]:
#Determine the unique values to inform what tags we will use
print(slope['avg_slope'].unique()) #min value of 0, max value 11
print(slope['max_slope'].unique()) #min value of 0, max value of 15
print(slope['min_slope'].unique()) #min value of 0, max value of 1

# Biodiversity 

The following cells are used to process the satoyama index dataset. This dataset must be requested via https://www.nies.go.jp/biology/data/si.html. Please allow 2 weeks for application processing. 

## Satoyama Biodiversity Score

The satoyama.csv file contains MESH2_ID identifiers with the following format [lat][lon][index_lat (0 - 7)][index_lon (0 - 7)]

For second order meshes, each tile is divided into 8 x 8 segments, each segment is 450 x 300 seconds or (7'30'') x (5')

The indices are fairly unintuitive, but a visual reference is available: https://www.nies.go.jp/biology/pdfs/lu_4.pdf

The code below converts from the MESH2_ID identifiers to decimal degrees. This code can also be tweaked to convert the MESH3_ID 3rd order dataset
 

In [ ]:
import geopandas as gpd

#Mapping function: Maps MESH2_ID indices to decimal degree
def map_to_degrees(c, a, b, x, y):
    return ((c - a) * (y - x) / (b - a)) + x

from shapely.geometry import Polygon
#Create geometry from decimal degrees and tile sizes

#Note, these tile sizes are specific to the 2nd order satoyama dataset
tile_size_lat = (5.0/60) #each tile is 1/8 of 40'
tile_size_lon = 0.125 #each tile is 1/8 of a degree

def generate_tile_polygon(upper_left_lat, upper_left_lon, tile_size_lat = tile_size_lat, tile_size_lon = tile_size_lon):
    upper_right_lat = upper_left_lat
    upper_right_lon = upper_left_lon + tile_size_lon

    lower_left_lat = upper_left_lat - tile_size_lat
    lower_left_lon = upper_left_lon

    lower_right_lat = upper_left_lat - tile_size_lat
    lower_right_lon = upper_left_lon + tile_size_lon

    return Polygon([
        (upper_left_lon, upper_left_lat),
        (upper_right_lon, upper_right_lat),
        (lower_right_lon, lower_right_lat),
        (lower_left_lon, lower_left_lat),
        (upper_left_lon, upper_left_lat)  # Closing the polygon
    ])
    
import pandas as pd
import numpy as np
def read_mesh_2ID(entry):
    parts = []
    #mapping is imprecise, I relied on the provided diagram and trial and error to get the following mappings
    #convert latitude
    lat_index = int(entry[0:2])
    lat = map_to_degrees(lat_index, 36, 71, 24.1, 47.4) #Based on the map, indices 36 yto 71 map to 24N 47.5N
    
    lon_index = int(entry[2:4]) #thankfully this index maps easily (30 -> 130 E)
    
    #Convert data indices to decimal degrees
    parts.append(lat + float(int(entry[4]) * tile_size_lat)) #latitude + tile_size * secondary index
    parts.append(100.0 + lon_index + float(int(entry[5]) * tile_size_lon)) #add 100 to index, longitude
  

    return pd.Series(parts)
    
#================================================================================================================================

shp = gpd.read_file("/home/janelle9/Japan-GIS-Data/biodiversity/satoyama.csv")
    
lat_lon = shp['MESH2_ID'].apply(lambda x: read_mesh_2ID(x)) #Convert indices to decimal degrees
polygons = shp.apply(lambda row: generate_tile_polygon(row['latitude'], row['longitude']), axis=1) #generate geometry from lat/lon values
shp['geometry'] = polygons
shp.crs = "EPSG:4326"

#After converting, we can save our shp
new_shp = shp[['latitude', 'longitude', 'geometry']]
new_shp['feature_id'] = new_shp.index #set index
new_shp.set_index('feature_id', inplace=True)

#we need to make satoyama values discrete
new_shp['satoyama'] = np.floor(pd.to_numeric(shp['MEAN'], errors='coerce') * 20.0)

#Our max and min values are 14.0 and 0.0 respectively, so we can use those for our tags
print("Max: ",  max(new_shp['satoyama']))
print("Min: ",  min(new_shp['satoyama']))

new_shp.to_file(output_directory + "biodiversity_tags")

# Sediment Analysis

## Combining multiple shapefiles

Used for processing sediment risk and slope shapefile datasets

In [ ]:
path = "/home/janelle9/Japan-GIS-Data/sediment_risk/"

#concatenates all shapefiles in specified directory
file = os.listdir(path)
path = [os.path.join(path, i) for i in file if ".shp" in i]
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)
gdf.crs = 'EPSG:4326'

gdf['feature_id'] = slope.index
gdf.set_index('feature_id', inplace=True)

gdf.to_file(output_directory + "sediment_combined_tags")

## Checking for invalid Geometries

In [ ]:
sediment = gpd.read_file("/home/janelle9/Japan-GIS-Data/natural_disaster/sediment_data_combined.shp")

# Check for invalid geometries
gdf = sediment
gdf['is_valid'] = gdf['geometry'].is_valid
invalid_geometries = gdf[~gdf['is_valid']]

# Attempt to fix invalid geometries using buffer(0)
gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.buffer(0) if not geom.is_valid else geom)

# Check again for any remaining invalid geometries
gdf['is_valid'] = gdf['geometry'].is_valid
remaining_invalid_geometries = gdf[~gdf['is_valid']]

# If there are still invalid geometries, handle them as needed
if not remaining_invalid_geometries.empty:
    print("There are still invalid geometries after attempting to fix them.")
    print(remaining_invalid_geometries)
    
    
#If the sediment data is valid, it can be used for positive/ negative sampling in a similar fashion to the PV locations data


# Combining Datasets

These cells are used to combine the geometry of existing datasets

In [14]:
#perform a join between two shapefiles
import geopandas as gpd

# Load the first shapefile (points, polygons, etc.)
shapefile1 = gpd.read_file(output_directory + "slope_aspect_tags")

# Load the second shapefile (polygons, typically the one containing larger areas)
shapefile2 = gpd.read_file(output_directory + "biodiversity_tags")

# Perform the spatial join based on 'within'
joined_data = gpd.sjoin(shapefile1, shapefile2, how='inner', op='within')

# Result will contain attributes from both shapefiles where shapefile1 is completely within shapefile2
joined_data

/home/janelle9/srai-tutorial/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


         feature_id_left  avg_slope  max_slope  min_slope  max_aspect  \
0                      0        3.0        6.0        0.0         3.0   
1                      1        2.0        6.0        1.0         3.0   
2                      2        2.0        4.0        0.0         6.0   
3                      3        1.0        3.0        0.0         2.0   
16                    16        2.0        4.0        1.0         7.0   
...                  ...        ...        ...        ...         ...   
5836924          5991702        2.0        2.0        0.0         2.0   
5836925          5991704        2.0        3.0        0.0         1.0   
5836926          5991705        2.0        2.0        0.0         2.0   
5836927          5991712        1.0        2.0        0.0         1.0   
5836928          5991713        2.0        3.0        0.0         2.0   

         min_aspect                                           geometry  \
0               4.0  POLYGON ((132.00000 34.00000

In [18]:
joined_data.to_file(output_directory + "slope_biodiversity_tags")

/tmp/ipykernel_185469/822811620.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  joined_data.to_file(output_directory + "slope_biodiversity_tags")
